Вашим заданием в этом модуле будет проанализировать и преобразовать данные о велопоездках клиентов компании Citi Bike (США), специализирующейся на прокате велосипедов.
Датасет представляет собой таблицу с информацией о 300 тысячах поездок за первые пять дней сентября 2018 года и включает в себя следующую информацию:

        starttime — время начала поездки (дата, время);
        stoptime — время окончания поездки (дата, время);
        start station id — идентификатор стартовой стоянки;
        start station name — название стартовой стоянки;
        start station latitude, start station longitude — географическая широта и долгота стартовой стоянки;
        end station id — идентификатор конечной стоянки;

        end station name — название конечной стоянки;
        end station latitude, end station longitude — географическая широта и долгота конечной стоянки;
        bikeid — идентификатор велосипеда;
        usertype — тип пользователя (Customer — клиент с подпиской на 24 часа или на три дня, Subscriber — подписчик с годовой арендой велосипеда);
        birth year — год рождения клиента;
        gender — пол клиента (0 — неизвестный, 1 — мужчина, 2 — женщина).



In [2]:
import pandas as pd
bike_data = pd.read_csv('data/citibike-tripdata.csv', sep =',')
bike_df = bike_data.copy()

In [3]:
bike_df.info()
# start station id missed


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   starttime                300000 non-null  object 
 1   stoptime                 300000 non-null  object 
 2   start station id         299831 non-null  float64
 3   start station name       299831 non-null  object 
 4   start station latitude   300000 non-null  float64
 5   start station longitude  300000 non-null  float64
 6   end station id           299831 non-null  float64
 7   end station name         299831 non-null  object 
 8   end station latitude     300000 non-null  float64
 9   end station longitude    300000 non-null  float64
 10  bikeid                   300000 non-null  int64  
 11  usertype                 300000 non-null  object 
 12  birth year               300000 non-null  int64  
 13  gender                   300000 non-null  int64  
dtypes: f

## Задание 6.3
Найдите идентификатор самой популярной стартовой стоянки. Запишите идентификатор в виде целого числа.

In [4]:
bike_df['start station id'].value_counts().nlargest(1)

281.0    1928
Name: start station id, dtype: int64

##  Задание 6.4
Велосипед с каким идентификатором является самым популярным?

In [5]:

bike_df['bikeid'].value_counts().nlargest(1)

33887    107
Name: bikeid, dtype: int64

##  Задание 6.5
Какой тип клиентов (столбец usertype) является преобладающим — Subscriber или Customer? В качестве ответа запишите долю клиентов преобладающего типа среди общего количества клиентов. Ответ округлите до сотых.

In [40]:
round(bike_df['usertype'].value_counts(normalize='True').max(), 2)

0.77

##  Задание 6.6 
Кто больше занимается велоспортом — мужчины или женщины? В ответ запишите число поездок для той группы, у которой их больше.

In [7]:
bike_df['gender'].value_counts().max()

183582

##  Задание 6.7
Выберите утверждения,которые соответствуют нашим данным:
 A: Число уникальных стартовых и конечных стоянок, которыми воспользовались клиенты, не равны друг другу
 B: В рассматриваемые дни минимальный возраст клиента составлял 10 лет
 C: Самой непопулярной стартовой стоянкой из тех, которыми воспользовались клиенты, является стоянка с названием "Eastern Pkwy & Washington Ave"
 D: Наибольшее количество поездок завершается на стоянке под названием "Liberty Light Rail" 

In [8]:
bike_df['starttime'] = pd.to_datetime(bike_df['starttime'])
display(bike_df['starttime'].dt.year.min())


2018

In [9]:
print('A:', bike_df['start station name'].unique() != bike_df['end station name'].unique())
print('B:', (bike_df['starttime'].dt.year.min() - bike_df['birth year'].max())<10)
print('C:', bike_df['start station name'].value_counts().index[-1] == "Eastern Pkwy & Washington Ave")
print('D:', bike_df['end station name'].value_counts().index[0] == "Liberty Light Rail")

A: True
B: False
C: True
D: False


/tmp/ipykernel_5385/682364967.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  print('A:', bike_df['start station name'].unique() != bike_df['end station name'].unique())


## Задание 6.8
В первую очередь удалим лишнюю информацию из данных.

В наших данных присутствуют столбцы, которые дублируют информацию друг о друге: это столбцы с идентификатором и названием стартовой и конечной стоянки. Удалите признаки идентификаторов стоянок. Сколько столбцов осталось?

In [10]:
bike_df.drop(['start station id', 'end station id'], axis = 1, inplace = True)

In [11]:
len(bike_df.columns)

12

##  Задание 6.9
Замените признак birth year на более понятный признак возраста клиента age. Годом отсчёта возраста выберите 2018 год. Столбец birth year удалите из таблицы. Сколько поездок совершено клиентами старше 60 лет?

In [12]:
bike_df['age'] = 2018 - bike_df['birth year']

In [41]:
bike_df.drop(['birth year'], axis=1, inplace=True)

In [42]:
display(bike_df[bike_df['age']>60].shape[0])

11837

##  Задание 6.10
Создайте признак длительности поездки trip duration. Для этого вычислите интервал времени между временем окончания поездки (stoptime) и временем её начала (starttime) в секундах. В качестве ответа запишите среднюю длительность поездки в секундах. Ответ округлите до целого.

Для того чтобы преобразовать временной интервал timedelta в секунды, используется атрибут dt.seconds (в datetime атрибут был second — в единственном числе).

In [14]:
bike_df['stoptime'] = pd.to_datetime(bike_df['stoptime'])
bike_df[['starttime', 'stoptime']]

,starttime,stoptime
0,2018-09-01 00:00:05.269,2018-09-01 00:27:20.634
1,2018-09-01 00:00:11.281,2018-09-01 00:02:23.481
2,2018-09-01 00:00:20.649,2018-09-01 00:55:58.547
3,2018-09-01 00:00:21.746,2018-09-01 00:07:38.583
4,2018-09-01 00:00:27.315,2018-09-01 02:21:25.308
...,...,...
299995,2018-09-05 19:08:27.846,2018-09-05 19:15:51.494
299996,2018-09-05 19:08:28.370,2018-09-05 19:20:01.508
299997,2018-09-05 19:08:27.509,2018-09-05 19:13:40.506
299998,2018-09-05 19:08:29.230,2018-09-05 20:04:29.322


In [44]:
#delta = bike_df['stoptime'] - bike_df['starttime']
#bike_df['trip duration'] = delta.dt.seconds
bike_df['trip duration2'] = (bike_df['stoptime'] - bike_df['starttime']).dt.seconds

#round(bike_df['trip duration'].mean())
round(bike_df['trip duration2'].mean())

1000

## Задание 6.11
Создайте «признак-мигалку» weekend, который равен 1, если поездка начиналась в выходной день (суббота или воскресенье), и 0 — в противном случае. Выясните, сколько поездок начиналось в выходные.

In [35]:
bike_df['weekend'] = bike_df['starttime'].dt.dayofweek.apply(lambda x: 1 if x in (5,6) else 0)
bike_df[bike_df['weekend'] == 1]['weekend'].shape[0]

115135

## Задание 6.12
Создайте признак времени суток поездки time_of_day. Время суток будем определять из часа начала поездки. Условимся, что:
* поездка совершается ночью (night), если её час приходится на интервал от 0 (включительно) до 6 (включительно) часов;
* поездка совершается утром (morning), если её час приходится на интервал от 6 (не включительно) до 12 (включительно) часов;
* поездка совершается днём (day), если её час приходится на интервал от 12 (не включительно) до 18 (включительно) часов;
* поездка совершается вечером (evening), если её час приходится на интервал от 18 (не включительно) до 23 часов (включительно).
Во сколько раз количество поездок, совершённых днём, больше, чем количество поездок, совёршенных ночью, за представленный в данных период времени? Ответ округлите до целых.

In [37]:
def time_of_day(hr):
    if 0<= hr <=6:
        return 'night'
    elif 6< hr <=12:
        return 'morning'
    elif 12< hr <=18:
        return 'day'
    elif 18< hr <=23:
        return 'evening'
    else:
        return None


bike_df['time_of_day'] = bike_df['starttime'].dt.hour.apply(time_of_day)  
round(bike_df[bike_df['time_of_day']=='day'].shape[0] / bike_df[bike_df['time_of_day']=='night'].shape[0])

9